In [2]:
from astropy.table import Table
import astropy.coordinates as coord
from astropy.coordinates import Galactocentric
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'

plt.rcParams.update({
    "text.usetex": True
})

import sys
sys.path.append("../")

from auridesi_helpers import rv_to_gsr, load, get_lsr_frame


In [3]:
iron = pd.read_csv("gmedinat/DESI-iron_RRLs_v0.2.csv")
iron

/var/folders/jn/p0g0f05x64b5yxz9ppdkcswh0000gn/T/ipykernel_61836/2730335157.py:1: DtypeWarning: Columns (103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  iron = pd.read_csv("gmedinat/DESI-iron_RRLs_v0.2.csv")


,source_id,designation,ra,dec,l,b,parallax,pmra,pmdec,phot_g_mean_mag,...,dist_from_metallicity,dist_from_metallicity_err,MG_from_FEH,MG_from_FEH_err,dist_from_FEH,dist_from_FEH_err,MG_from_feh_k_mean,MG_from_feh_k_err,dist_from_feh_k_mean,dist_from_feh_k_err
0,1043873010812474752,Gaia DR3 1043873010812474752,133.382084,63.113076,152.414181,37.669321,0.025369,1.784098,-1.774047,18.177101,...,NaN,NaN,0.066183,0.073302,37.844405,1.285579,0.339014,0.065577,33.376097,1.015889
1,1043938427457808384,Gaia DR3 1043938427457808384,132.811179,63.342705,152.213143,37.363655,-0.048998,1.646551,-1.953840,18.390121,...,36.514716,1.655250,0.073858,0.073783,39.206708,1.332866,0.177314,0.125849,37.382578,2.166918
2,4421572696984276992,Gaia DR3 4421572696984276992,229.693758,2.103003,3.934386,46.767093,0.062328,4.077584,-9.838028,14.812551,...,6.000626,0.404481,0.584537,0.041868,6.631684,0.130968,0.700182,0.041627,6.287743,0.123494
3,4421572456466029696,Gaia DR3 4421572456466029696,229.729180,2.097915,3.960993,46.736327,0.088767,4.059161,-9.884413,15.024820,...,7.121420,0.712811,0.637500,0.040505,6.783998,0.177621,0.703721,0.048678,6.580236,0.190724
4,4421569982565299200,Gaia DR3 4421569982565299200,229.831329,2.084621,4.039123,46.648433,0.166676,4.210531,-9.889124,14.976509,...,NaN,NaN,0.630791,0.040656,7.018571,0.158363,0.639798,0.043257,6.989517,0.164721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6082,3936893742228790528,Gaia DR3 3936893742228790528,199.504811,16.814436,335.102810,77.991892,0.042649,-1.476322,-0.568054,18.215242,...,NaN,NaN,0.454147,0.062761,34.467661,0.997936,0.524964,0.054719,33.361708,0.842598
6083,3936953979146081792,Gaia DR3 3936953979146081792,198.664896,17.196890,332.512042,78.711663,0.077993,-6.765189,-3.640127,15.691649,...,10.945668,0.591894,0.268659,0.057106,11.076496,0.295393,0.356444,0.054765,10.637641,0.272392
6084,3936427171341527552,Gaia DR3 3936427171341527552,197.613731,16.293353,326.035991,78.303103,-0.130831,-1.067740,-0.762754,18.617080,...,40.304636,1.710767,0.379188,0.057075,42.021546,1.112807,0.394160,0.077587,41.732811,1.497191
6085,3938945671444754432,Gaia DR3 3938945671444754432,199.618671,19.119141,342.134825,79.866787,-0.048671,-0.510533,-0.958867,18.870989,...,52.139997,4.384021,-0.022500,0.107317,55.336340,2.754791,0.309348,0.056999,47.494244,1.278705


In [4]:
RRL_coords = coord.SkyCoord(
    ra=iron['ra'].to_list() * u.deg, dec=iron['dec'].to_list() * u.deg,
    distance=iron['dist_constMet'].to_list()*u.kpc,
    pm_ra_cosdec=(iron['pmra'].to_numpy() * np.cos(iron['dec'].to_numpy()*np.pi/180))*u.mas/u.yr,
    pm_dec=iron['pmdec'].to_list()*u.mas/u.yr,
    radial_velocity=iron['v0_mean'].to_list()*u.km/u.s
)

V_GSR = np.array(rv_to_gsr(RRL_coords))
iron['V_GSR'] = V_GSR
iron['high_V_GSR'] = np.abs(iron['V_GSR']) > 400


In [5]:
print("Total number of RRL (Medina v0.2): ", len(iron))
print("Total number SN_R<3: ", len(iron[iron['SN_R'] < 3]))
print("Total number in dwarf: ", len(iron[iron['in_gc_dwarf'] == 'dwarf']))
print("Total number in gc: ", len(iron[iron['in_gc_dwarf'] == 'gc']))
print("Total number in |V_GSR| > 400: ", len(iron[np.abs(iron['V_GSR']) > 400]))
print("Total number with nan ra", np.sum(iron['ra'].isnull()))
print("Total number with nan dec", np.sum(iron['dec'].isnull()))
print("Total number with nan v0_mean", np.sum(iron['v0_mean'].isnull()))

# np.abs(rcat['vT']) < 500, vT is azimuthal


Total number of RRL (Medina v0.2):  6087
Total number SN_R<3:  1
Total number in dwarf:  62
Total number in gc:  22
Total number in |V_GSR| > 400:  0
Total number with nan ra 0
Total number with nan dec 0
Total number with nan v0_mean 0


In [6]:
iron = iron[
    (iron['SN_R'] > 3) &
    (iron['in_gc_dwarf'] != 'dwarf') &
    (iron['in_gc_dwarf'] != 'gc') &
    (np.abs(iron['V_GSR']) < 400) &
    (~iron['ra'].isnull()) &
    (~iron['dec'].isnull()) &
    (~iron['v0_mean'].isnull())
]

print(len(iron), "remaining RRL")


6002 remaining RRL


In [7]:
iron.to_csv("gmedinat/DESI-iron_RRLs_v0.2_qualcut.csv", index=None)